In [4]:
import numpy as np
from glob import glob
from PIL import Image
from sklearn import metrics
from skimage.metrics import variation_of_information
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch

In [5]:
def calculate_overlap(r1, r2):
    # intersection
    a = np.count_nonzero(r1 * r2)
    # union
    b = np.count_nonzero(r1 + r2)
    return a/b

def calculate_SC(segmentation, gt):
    N = segmentation.shape[0] * segmentation.shape[1]
    maxcoverings_sum = 0
    for label1 in np.unique(segmentation):
        # region1 is where the segmentation has a specific label
        region1 = (segmentation == label1).astype(int) 
        # |R| is the size of nonzero elements as this the region size
        len_r = np.count_nonzero(region1) 
        max_overlap = 0
        # Calculate max overlap 
        for label2 in np.unique(gt):
            # region2 is where the segmentation has a specific label
            region2 = (gt == label2).astype(int)
            # Calculate overlap
            overlap = calculate_overlap(region1, region2)
            max_overlap = max(max_overlap, overlap)
        maxcoverings_sum += (len_r * max_overlap)
    return (1 / N) * maxcoverings_sum

def calculate_VI(segmentation, gt):
    VI = variation_of_information(segmentation, gt)
    return VI

def calculate_PRI(segmentation, gt):
    PRI = metrics.adjusted_rand_score(segmentation.flatten(), gt.flatten())
    return PRI

def calculate_mIOU(segmentation, gt):
    gt_array = gt
    miou_for_each_class = []
    label_list = np.unique(gt_array)

    # gt_mask is 0 where gt label is 0 (background) and 1 where gt label is not 0 (foreground)
    gg = np.zeros(gt_array.shape)
    gt_mask = np.where(gt_array > 0, 1, gg)
    # label_list is the list of labels in the input array (range(k))
    label_list = np.unique(gt_array)


    gt_array_1d = gt.reshape((gt.shape[0]*gt.shape[1])) # 1d array of gt labels
    input_array_1d = segmentation.reshape((segmentation.shape[0]*segmentation.shape[1])) # 1d array of input labels

    miou_list = []
    # For each class in range k
    for l in label_list:
        inds = np.where( gt_array_1d == l )[0] # indices of gt where label is l
        pred_labels = input_array_1d[ inds ] # predictions at those indices
        u_pred_labels = np.unique(pred_labels) # unique predictions at those indices
        hists = [ np.sum(pred_labels == u) for u in u_pred_labels ] # frequency of each unique prediction at those indices
        fractions = [ len(inds) + np.sum(input_array_1d == u) - np.sum(pred_labels == u) for u in u_pred_labels ] # (total number of pixels in gt with label l) + (total number of pixels in input with label u) - (total number of pixels in input with label u and gt label l)
        mious = hists / np.array(fractions,dtype='float')
        miou_list.append( np.max(mious) )
        miou_for_each_class.append( np.max(mious) )
        
    mIOU = sum(miou_list) / float(len(miou_list))

    return mIOU

In [11]:
# Here
def evaluate(method_name):
    print(f"Evaluating {method_name}...")
    with open("datasets/noise/test_ids.txt", "r") as f:
        noise_segmentation_paths = [f"results/{method_name}/noise/" + line.strip() + ".png" for line in f.readlines()]
    with open("datasets/clouds/test_ids.txt", "r") as f:
        cloud_segmentation_paths = [f"results/{method_name}/clouds/" + line.strip() + ".png" for line in f.readlines()]
    with open("datasets/texture/test_ids.txt", "r") as f:
        texture_segmentation_paths = [f"results/{method_name}/texture/" + line.strip() + ".png" for line in f.readlines()]
    # segmentation_times = []
    cloud_scs = []
    cloud_vis = []
    cloud_pris = []
    cloud_mious = []
    print(f"\tclouds...")
    for path in tqdm(cloud_segmentation_paths):
        id = path.split("/")[-1].split(".")[0]
        segmentation = Image.open(path)
        # segmentation_time = float(segmentation.text["segmentation_time"])
        # segmentation_times.append(segmentation_time)
        segmentation = np.array(segmentation)
        gt = np.array(Image.open(f"datasets/clouds/gt/{id}.png"))
        segmentation = torch.from_numpy(segmentation).unsqueeze(0).unsqueeze(0)
        segmentation = torch.nn.functional.interpolate(segmentation, size=gt.shape, mode="nearest").squeeze().numpy()
        sc = calculate_SC(segmentation, gt)
        vi = calculate_VI(segmentation, gt)
        pri = calculate_PRI(segmentation, gt)
        miou = calculate_mIOU(segmentation, gt)
        cloud_scs.append(sc)
        cloud_vis.append(vi)
        cloud_pris.append(pri)
        cloud_mious.append(miou)
    noise_scs = {"0.5":[], "1.0":[], "2.0":[]}
    noise_vis = {"0.5":[], "1.0":[], "2.0":[]}
    noise_pris = {"0.5":[], "1.0":[], "2.0":[]}
    noise_mious = {"0.5":[], "1.0":[], "2.0":[]}
    print(f"\tnoise...")
    for path in tqdm(noise_segmentation_paths):
        id = path.split("/")[-1].split(".")[0]
        noise_level = path.split("/")[-2]
        segmentation = Image.open(path)
        # segmentation_time = float(segmentation.text["segmentation_time"])
        # segmentation_times.append(segmentation_time)
        segmentation = np.array(segmentation)
        gt = np.array(Image.open(f"datasets/noise/gt/{id}.png").convert("L"))
        segmentation = torch.from_numpy(segmentation).unsqueeze(0).unsqueeze(0)
        segmentation = torch.nn.functional.interpolate(segmentation, size=gt.shape, mode="nearest").squeeze().numpy()
        sc = calculate_SC(segmentation, gt)
        vi = calculate_VI(segmentation, gt)
        pri = calculate_PRI(segmentation, gt)
        miou = calculate_mIOU(segmentation, gt)
        noise_scs[noise_level].append(sc)
        noise_vis[noise_level].append(vi)
        noise_pris[noise_level].append(pri)
        noise_mious[noise_level].append(miou)
    texture_scs = []
    texture_vis = []
    texture_pris = []
    texture_mious = []
    print(f"\ttexture...")
    for path in tqdm(texture_segmentation_paths):
        id = f'{int(path.split("/")[-1].split(".")[0].split("_")[0]):04d}'
        segmentation = Image.open(path)
        # segmentation_time = float(segmentation.text["segmentation_time"])
        # segmentation_times.append(segmentation_time)
        segmentation = np.array(segmentation)
        gt = np.array(Image.open(f"datasets/texture/gt/{id}.png").convert("L"))
        segmentation = torch.from_numpy(segmentation).unsqueeze(0).unsqueeze(0)
        segmentation = torch.nn.functional.interpolate(segmentation, size=gt.shape, mode="nearest").squeeze().numpy()
        sc = calculate_SC(segmentation, gt)
        vi = calculate_VI(segmentation, gt)
        pri = calculate_PRI(segmentation, gt)
        miou = calculate_mIOU(segmentation, gt)
        texture_scs.append(sc)
        texture_vis.append(vi)
        texture_pris.append(pri)
        texture_mious.append(miou)

    results = {
        "cloud": {
            "sc": np.mean(cloud_scs),
            "vi": np.mean(cloud_vis),
            "pri": np.mean(cloud_pris),
            "miou": np.mean(cloud_mious)
        },
        "noise_0.5": {
            "sc": np.mean(noise_scs["0.5"]),
            "vi": np.mean(noise_vis["0.5"]),
            "pri": np.mean(noise_pris["0.5"]),
            "miou": np.mean(noise_mious["0.5"])
        },
        "noise_1.0": {
            "sc": np.mean(noise_scs["1.0"]),
            "vi": np.mean(noise_vis["1.0"]),
            "pri": np.mean(noise_pris["1.0"]),
            "miou": np.mean(noise_mious["1.0"])
        },
        "noise_2.0": {
            "sc": np.mean(noise_scs["2.0"]),
            "vi": np.mean(noise_vis["2.0"]),
            "pri": np.mean(noise_pris["2.0"]),
            "miou": np.mean(noise_mious["2.0"])
        },
        "texture": {
            "sc": np.mean(texture_scs),
            "vi": np.mean(texture_vis),
            "pri": np.mean(texture_pris),
            "miou": np.mean(texture_mious)
        },
        "all": {
            "sc": np.mean(cloud_scs + noise_scs["2.0"] + texture_scs),
            "vi": np.mean(cloud_vis + noise_vis["2.0"] + texture_vis),
            "pri": np.mean(cloud_pris + noise_pris["2.0"] + texture_pris),
            "miou": np.mean(cloud_mious + noise_mious["2.0"] + texture_mious)
        }
    }
    return results

results = {
    "DFC": evaluate("DFC"),
    "GNEMS": evaluate("GNEMS"),
    "DSS": evaluate("DSS")
}



Evaluating DFC...
results/DFC/noise/0.5/0000.png
results/DFC/clouds/0003.png
results/DFC/texture/118_1.png
Evaluating GNEMS...
results/GNEMS/noise/0.5/0000.png
results/GNEMS/clouds/0003.png
results/GNEMS/texture/118_1.png
Evaluating DSS...
results/DSS/noise/0.5/0000.png
results/DSS/clouds/0003.png
results/DSS/texture/118_1.png


In [8]:
# pretty print results
print("Method\t\t\tSC\t\tVI\t\tPRI\t\tmIOU")
for method in results:
    print(f"{method}\t\t{results[method]['all']['sc']:.3f}\t\t{results[method]['all']['vi']:.3f}\t\t{results[method]['all']['pri']:.3f}\t\t{results[method]['all']['miou']:.3f}")


Method			SC		VI		PRI		mIOU
DFC		0.605		0.661		0.437		0.651
GNEMS		0.753		0.445		0.502		0.726
DSS		0.455		0.882		0.053		0.432
